In [ ]:
import sys
from pathlib import Path
import os
cwd = os.getcwd()
parent = str(Path(cwd).parents[0])
sys.path.append(parent)

In [ ]:
import pandas as pd
import numpy as np
import collections

In [ ]:
arms = pd.read_csv('https://raw.githubusercontent.com/maia-sh/direcct-analysis/main/data/reporting/arms.csv')

In [ ]:
len(arms)

In [ ]:
intervention_arms = arms[arms.type == 'experimental']

In [ ]:
intervention_arms.head()

In [ ]:
included_trials = pd.read_csv('https://raw.githubusercontent.com/maia-sh/direcct-analysis/main/data/reporting/main-analyses/kaplan-meier-time-to-pub.csv')

In [ ]:
included_trials.head()

In [ ]:
intervention_arms2 = intervention_arms[intervention_arms.id.isin(included_trials.id.to_list())].reset_index(drop=True)

In [ ]:
len(intervention_arms2)

In [ ]:
grouped = intervention_arms2[['id', 'intervention']].groupby('id')['intervention'].apply(list)

In [ ]:
ids = grouped.index.to_list()
ints = grouped.to_list()

In [ ]:
unique_tx_dict = {}
running_int_list = []

for i, t in zip(ids, ints):
    all_ints = []
    for x in t:
        if ';' in x:
            split_ints = x.split(';')
            all_ints = all_ints + split_ints
        else:
            all_ints.append(x)
    unique_tx_dict[i] = ';'.join(set(all_ints))
    running_int_list = running_int_list + list(set(all_ints))

In [ ]:
c = collections.Counter(x for x in running_int_list if x)

In [ ]:
int_counts = pd.DataFrame.from_dict(dict(c), orient='index').reset_index()
int_counts.columns = ['interventions', 'counts']

In [ ]:
int_counts.sort_values(by='counts', ascending=False).to_csv(parent + '/data/interventions/' + 'top_ints.csv')

In [ ]:
int_counts.sort_values(by='counts', ascending=False).head(10)

In [ ]:
pd.DataFrame.from_dict(unique_tx_dict, orient='index').reset_index().rename({'index':'id', 0:'intervention'}, axis=1).to_csv(parent + '/data/interventions/' + 'int_mapping.csv')